In [1]:
conda install -c conda-forge ffmpeg

Note: you may need to restart the kernel to use updated packages.




EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - ffmpeg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.13.0               |   py38haa244fe_1        1016 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        1016 KB

The following packages will be UPDATED:

  conda                               4.11.0-py38haa244fe_0 --> 4.13.0-py38haa244fe_1




conda-4.13.0         | 1016 KB   |            |   0% 
conda-4.13.0         | 1016 KB   | 1          |   2% 
conda-4.13.0         | 1016 KB   | ####4      |  44% 
conda-4.13.0         | 1016 KB   | ########## | 100% 
conda-4.13.0         | 1016 KB   | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... failed


In [2]:
%matplotlib qt

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from itertools import count


"""
f = 0.04, k = 0.06, r_u = 0.01, r_v = 0.005
"""
def laplacien(mesh,dx):
    """ pour dx = dy (maille carrée), caclul du laplacien"""
    laplx = (np.roll(mesh,1,axis = 1)-2*mesh + np.roll(mesh,-1,axis = 1))/(dx**2)
    laply = (np.roll(mesh,1,axis = 0)-2*mesh + np.roll(mesh,-1,axis = 0))/(dx**2)
    return(laplx+laply)

dt = 0.01
T = 5
t = np.linspace(0,T,int(np.round(T/dt))) #échelle de temps de simulation
grille = 100 #taille de grille de simulation
Longueur_phys = 10**-3 #taille physique de la grille, en m
Du = 10**-9 # coeff diffusion en phase liquide (eau-éthanol, en odg SI)

x = np.linspace(1,grille,grille)*Longueur_phys/grille
y = np.linspace(1,grille,grille)*Longueur_phys/grille
X, Y = np.meshgrid(x,y) #grille de position pour le calcul

#initialisation des concentrations en u et v :
U = np.zeros((100,100))
U[25:45,43:47]=1
U[55:75,53:57]=1
V = np.zeros((100,100))

store_u = [U]
store_v = [V]
for inst in range(len(t)):
    U = U+dt*Du*(laplacien(U,Longueur_phys/grille))
    store_u.append(U)

fig1,ax1 = plt.subplots()
pos = ax1.imshow(store_u[0])
newcol = fig1.colorbar(pos, ax=ax1)

def updatefig(i):
    #print(i)
    if i+1 < len(store_u) :
        pos.set_data(store_u[i+1])
    else : 
        pos.set_data(store_u[-1])#on affiche la dernière frame plutôt que de planter
    newcol

ani = animation.FuncAnimation(fig1, updatefig, frames = range(len(store_u)), interval=25,repeat = True) #intervalle de 25 ms entre les frames
print("got here")


HTML(ani.to_html5_video())

got here


Modèle de Gray Scott

In [3]:
%matplotlib qt

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from itertools import count


"""
f = 0.04, k = 0.06, r_u = 0.01, r_v = 0.005
"""
def laplacien(mesh,dx):
    """ pour dx = dy (maille carrée), caclul du laplacien"""
    laplx = (np.roll(mesh,1,axis = 1)-2*mesh + np.roll(mesh,-1,axis = 1))/(dx**2)
    laply = (np.roll(mesh,1,axis = 0)-2*mesh + np.roll(mesh,-1,axis = 0))/(dx**2)
    return(laplx+laply)

dt = 0.02
T = 100
t = np.linspace(0,T,int(np.round(T/dt))) #échelle de temps de simulation
grille = 200 #taille de grille de simulation
Longueur_phys = 2*10**-3 #taille physique de la grille, en m
Du = 1*10**-9 # coeff diffusion en phase liquide (eau-éthanol, en odg SI)
Dv = 1*10**-9
facteur = 2 #facteur d'échelle de normalisationn des constantes de nutrition/décès fac not equal to 1
kcatalyse = 1*facteur
kdeces = 0.1*facteur
apport= 0.035*facteur

x = np.linspace(1,grille,grille)*Longueur_phys/grille
y = np.linspace(1,grille,grille)*Longueur_phys/grille
X, Y = np.meshgrid(x,y) #grille de position pour le calcul

#initialisation des concentrations en u et v :
U = np.zeros((100,100))
U[25:35,43:47]=1
U[21:25,23:53]=1
U[55:75,53:57]=1
V = np.ones((100,100))*1

store_u = [U]
store_v = [V]

for inst in range(len(t)):
    U = U+dt*Du*(laplacien(U,Longueur_phys/grille)) + kcatalyse*V*(U**2)*dt - kdeces*U*dt
    V = V+dt*Dv*(laplacien(V,Longueur_phys/grille)) - kcatalyse*V*(U**2)*dt + dt*apport*(1-U)
    store_u.append(U)
    store_v.append(V)

fig1,(ax1,ax2) = plt.subplots(1,2)
pos = ax1.imshow(store_u[0])
newcol = fig1.colorbar(pos, ax=ax1)
pos2 = ax2.imshow(store_v[0])
newcol2 = fig1.colorbar(pos2, ax=ax2)

def updatefig(i):
    #print(i)
    if i+1 < len(store_u) :
        pos.set_data(store_u[i+1])
        pos2.set_data(store_v[i+1])
    else : 
        pos.set_data(store_u[-1])#on affiche la dernière frame plutôt que de planter
        pos2.set_data(store_v[-1])
    newcol
    newcol2

ani = animation.FuncAnimation(fig1, updatefig, frames = range(len(store_u)), interval=25,repeat = True) #intervalle de 25 ms entre les frames



HTML(ani.to_html5_video())

KeyboardInterrupt: 

In [ ]:
print(np.ones((3,3)))

In [ ]:
print(np.roll(a,1,axis = 0))

In [ ]:
#-----------------------------------------------------------------------
# Ondes thermiques dans le sol
#-----------------------------------------------------------------------
# Renseignements/bugs : Guillaume Dewaele <agreg(at)sci-phy.org>
#-----------------------------------------------------------------------
#ligne affichant les plots dans un nouvel onglet
%matplotlib qt

# Paramètres modifiables

# rho * cp du sol
rhocp = 2.0e6

# Vitesse de l'animation (intervalle entre images en ms)
dt = 25

# Profondeur max
dmax = 5.0

#-----------------------------------------------------------------------

# Bibliothèques utilisées

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from matplotlib.colors import ListedColormap
import matplotlib.widgets as mwg
from itertools import count
import matplotlib.pylab as plb


#-----------------------------------------------------------------------

memParams = plt.rcParams

plt.rcParams.update(
  { 'legend.fontsize' : 'medium',
    'axes.labelsize' : 'medium',
    'axes.titlesize' : 'medium',
    'xtick.labelsize' : 'medium',
    'ytick.labelsize' : 'medium',
    'font.size' : 10 })

# Détection utilisation hors Pyzo

if '__iep__' not in globals() :
    matplotlib.interactive(False)
matplotlib.interactive(True)
#-----------------------------------------------------------------------


WT = np.linspace(0, 2*np.pi, 200)
Z = np.linspace(-dmax, 0, 200)

MWT, MZ = np.meshgrid(WT, Z)

T = np.zeros(MZ.shape)

def Update(_) :
    global delta
    
    A = slider_A.val
    lambd = slider_L.val
    omega = 2*np.pi/(86400*slider_T.val)
    
    delta = ((2*lambd)/(rhocp*omega))**0.5
    
    T[:] = A*np.exp(MZ/delta)*np.sin(MWT+MZ/delta)
    
    onde.set_data(T[::-1,:])
    onde.set_extent([0, slider_T.val, -dmax, 0])
    ax2.set_xlim(0, slider_T.val)
    
    t1.set_text("c={}m/j".format(round(omega*delta*86400,2)))
    t2.set_text("$\delta$={}m".format(round(delta,2)))
    
    fig.canvas.draw()
    
def Set(i) :
    i = i%len(T[0])
    sol.set_data(T[::-1,i:i+1])
    th1.set_ydata([-20, 15+T[-1,i]])
    th2.set_ydata([-20, 15+T[0,i]])
    
    return [ sol, th1, th2, t1, t2 ]
    
fig = plt.figure("Ondes thermiques dans le sol", figsize=(20,11))

ax1 = fig.add_axes([0.1, 0.22, 0.2, 0.7])
sol = ax1.imshow(T[::-1,0:1], extent=[0, 1, Z[0], Z[-1]], vmin=-15, vmax=15)
ax1.axis("auto")
ax1.set_xticks([])
ax1.set_xlim(0, 1)
ax1.set_ylim(Z[0], Z[-1])
ax1.set_ylabel("Profondeur (m)")
ax1.set_title("Onde thermique")

t1 = ax1.text(0.2,-0.82*dmax, r"c=0m/s", fontsize=14, color="LightYellow")
t2 = ax1.text(0.2,-0.92*dmax, r"$\delta$=0m", fontsize=14, color="LightYellow")

ax1a = fig.add_axes([0.38, 0.22, 0.02, 0.7])
th1, = ax1a.plot([0,0], [-10, 25], color="Red", linewidth=8)
ax1a.grid()
ax1a.set_title(r"T surf.")
ax1a.axis("auto")
ax1a.set_xticks([])
ax1a.set_xlim(-1, 1)
ax1a.set_ylim(-9, 39)

ax1b = fig.add_axes([0.45, 0.22, 0.02, 0.7])
th2, = ax1b.plot([0,0], [-10, 25], color="Red", linewidth=8)
ax1b.grid()
ax1b.set_title(r"T -{}m".format(dmax))
ax1b.axis("auto")
ax1b.set_xticks([])
ax1b.set_xlim(-1, 1)
ax1b.set_ylim(-9, 39)

ax2 = fig.add_axes([0.55, 0.22, 0.4, 0.7])
onde = ax2.imshow(T[::-1,:], extent=[WT[0], WT[-1], Z[0], Z[-1]], vmin=-15, vmax=15)
ax2.axis("auto")
ax2.set_xlim(WT[0], WT[-1])
ax2.set_ylim(Z[0], Z[-1])
ax2.set_xlabel("t (jours)")

axTmp = plt.axes([0.05, 0.05, 0.22, 0.025])
axTmp.set_title("Amplitude (°C)")
slider_A = mwg.Slider(axTmp, '', valmin=1, valmax=22, valinit=15, color='r', alpha=0.6)
slider_A.on_changed(Update)

axTmp = plt.axes([0.35, 0.05, 0.22, 0.025])
axTmp.set_title("Période (jours)")
slider_T = mwg.Slider(axTmp, '', valmin=1, valmax=365, valinit=365, color='b', alpha=0.6)
slider_T.on_changed(Update)

axTmp = plt.axes([0.65, 0.05, 0.22, 0.025])
axTmp.set_title("Conductivité thermique (W/mK)")
slider_L = mwg.Slider(axTmp, '', valmin=0.2, valmax=5.0, valinit=1.5, color='y', alpha=0.6)
slider_L.on_changed(Update)

Update(None)

anim = ani.FuncAnimation(fig, Set, count(0), interval=dt, blit=True)

#-----------------------------------------------------------------------

plb.rcParams.update(memParams)



In [ ]:

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from itertools import count


"""
f = 0.04, k = 0.06, r_u = 0.01, r_v = 0.005
"""
def laplacien(mesh,dx):
    """ pour dx = dy (maille carrée), caclul du laplacien"""
    
    #normalisation temporaire 
    #dx = 1
    
    laplx = (np.roll(mesh,1,axis = 1)-2*mesh + np.roll(mesh,-1,axis = 1))/(dx**2)
    laply = (np.roll(mesh,1,axis = 0)-2*mesh + np.roll(mesh,-1,axis = 0))/(dx**2)
    return(laplx+laply)


for appo in [0.0305]:
    for facteur in [1]:
        dt = 1 #en une seconde
        T =1000
        t = np.linspace(0,T,int(np.round(T/dt))) #échelle de temps de simulation
        grille = 25 #taille de grille de simulation
        
        #normalisation temporaire : 
        #Longueur_phys = 1*1*10**-3 #taille physique de la grille, en m
        Longueur_phys = 1
        Longueur_phys = grille*1e-5 #normalisé à 10µm par pixel 
        
        #Du = 3*10**-9 # coeff diffusion en phase liquide (eau-éthanol, en odg SI)
        #Dv = 0.1*10**-9
        #normalisation temporaire :
        Du = 0.06
        Dv = 0.03
        
        #dimension physique : 
        Du = 6*1e-12 #glucose
        Dv = 3*1e-12 #ATP
            
        #facteur = 2.5 #facteur d'échelle de normalisationn des constantes de nutrition/décès fac not equal to 1
        kcatalyse = 1/dt #inchangé si dt inchangé?
        kdeces = 0.062/dt
        fapport= appo
        
        x = np.linspace(1,grille,grille)*Longueur_phys/grille
        y = np.linspace(1,grille,grille)*Longueur_phys/grille
        X, Y = np.meshgrid(x,y) #grille de position pour le calcul
        
        #initialisation des concentrations en u et v :
        """
            U = np.zeros((100,100))
        U[25:35,43:47]=1
        U[21:25,23:53]=1
        U[55:75,53:57]=1
        """
        #U = np.random.rand(grille,grille)
        V = np.zeros((grille,grille))
        #+ np.random.rand(grille,grille)*0.05
        
        
        V[10:15,4:11]= 0.25+0.02*np.random.rand(5,7)
        #V[45:50,44:51]= 0.25+0.02*np.random.rand(5,7)
        
        #U[140:144,140:146]=0.25
        #U[143:144,142:149]=0.25
        #U[31:35,23:33]=0.5
        #U[131:135,123:133]=0.5
        
        
        U = np.ones((grille,grille))
        #ÆU[45:50,44:51]=0.5
        U[5:15,4:16]=0.5
        
        fig1,(ax1,ax2) = plt.subplots(1,2)
        pos = ax1.imshow(U)
        newcol = fig1.colorbar(pos, ax=ax1)
        pos2 = ax2.imshow(V)
        newcol2 = fig1.colorbar(pos2, ax=ax2)
        
        def updatefig(i):
            global U
            global V
            #print(i)
            pos.set_data(U)
            pos2.set_data(V)
            #U = np.where(U > 10**3, 1, U)
            U = U+dt*Du*(laplacien(U,Longueur_phys)) - U*(V**2)*dt + dt*fapport*(1-U)
            V = V+dt*Dv*(laplacien(V,Longueur_phys)) + U*(V**2)*dt - (kdeces+fapport)*V*dt

        
        ani = animation.FuncAnimation(fig1, updatefig, frames = range(len(t)), interval=2,repeat = True) #intervalle de 25 ms entre les frames
        ani.save('simgrayefficace6.mp4'.format(facteur,appo), writer="ffmpeg")
        